## cleaned_df Pre-Processing 

### Khai báo một số thư viện sẽ sử dụng trong quá trình xử lí. 

In [1]:
import pandas as pd 
import numpy as np
import sys  
import re 


In [2]:
!pwd 

/home/huylee/DataScience/Final_project


### Đầu tiên, ta tiến hành đọc dữ liệu từ file 

In [3]:
raw_df = pd.read_csv('DATA/infant_mortality_data_expanded_2017_2022.csv', sep=',', quotechar='"', header=0)

In [4]:
raw_df.head()

,Cause of death,NICU Admission,Antibiotics for Mother,Mother's Pre-pregnancy BMI,Infant birth weight,Deaths,Births,Death rate per 1000,Year
0,Other and unspecified gastroenteritis and coli...,Yes,Yes,Overweight 25.0-29.9,500 - 749 grams,12.0,726.0,16.56 (Unreliable),2017
1,Other and unspecified gastroenteritis and coli...,Yes,No,Normal 18.5-24.9,500 - 749 grams,13.0,1254.0,10.40 (Unreliable),2017
2,Other and unspecified gastroenteritis and coli...,Yes,No,Normal 18.5-24.9,750 - 999 grams,10.0,1701.0,5.88 (Unreliable),2017
3,Other and unspecified gastroenteritis and coli...,Yes,No,Overweight 25.0-29.9,500 - 749 grams,12.0,994.0,12.08 (Unreliable),2017
4,Newborn affected by incompetent cervix,No,Yes,Normal 18.5-24.9,499 grams or less,12.0,270.0,45.00 (Unreliable),2017


### Số lượng cột và hàng của mẫu. 

In [5]:
shape = raw_df.shape

In [6]:
# TESTING: 
print(f'Số lượng dòng và cột: {shape}')

if shape[0] > 1000:
    print("số lượng mẫu đạt yêu cầu")
else: 
    print("số lượng mẫu không đạt yêu cầu, ta cần thêm dữ liệu")



Số lượng dòng và cột: (1443, 9)
số lượng mẫu đạt yêu cầu


### Mẫu có tồn tại trùng lặp không ? 
- sau khi ta đã xác định có số lượng mẫu đủ để tiếp tục xử lí thì tiến hành kiểm tra sự trung lặp `num_duplicated_rows` để giảm bớt các mẫu không cần thiết.  

In [7]:
num_duplicates = raw_df.duplicated().sum()

In [8]:
# TESTING:
if num_duplicates > 0:
    print(f'Số lượng dòng bị trùng: {num_duplicates}')
    print("Bạn cần xử lý dữ liệu trùng lặp")
else: 
    print("Không có dòng nào bị trùng")

Không có dòng nào bị trùng


sau khi kiểm tra ta thấy dữ liệu không còn trùng lặp ta tiếp tục đi sau hơn vào từng cột.  

### Các cột trong mẫu có ý nghĩa như thế nào?  
Để kiểm tra ý nghĩa của từng cột: 
- Ta lấy của các cột thông tin từ `raw_`. 
- Do chưa được mã hóa nên cột mang ý nghĩa của chính nó mà không phải thông qua giải thích.  
- Tiếp đó ta `hiển thị` các cột được lấy lên màn hình để quan sát. 

In [ ]:
col_mean = pd.DataFrame(raw_df.columns, columns=['Meaning'])

AttributeError: module 'pandas' has no attribute 'Frame'

In [ ]:
col_mean

Tại đây ta nên dừng lại một chút để đọc lại và hiểu kĩ hơn về các cột. Đồng thời chúng ta có thể nhận thấy được các cột trên sẽ có thể mang những kiểu dữ liệu, cách biểu diễn phù hợp. 

### Kiểu dữ liêu của từng cột là gì? Dữ liều này có hợp lý hay không, có cần phải chuyển đổi các dữ liệu không? 

In [ ]:
dtypes = raw_df.dtypes

In [ ]:
dtypes

#### Ta tiến hành xem lại và xử lí từng côt dữ liệu nếu chưa hợp lí. 

In [ ]:
raw_df.head(10)

Sau khi quan sát lại thì ta thấy dữ liệu thu thập được đa phần ở dạng `text` và không thể xử lí trực tiếp được. Vì vậy ta cần chuyển đổi nó về các dạng/kiểu dữ liệu tốt hơn để tiến hành lưu trữ, phù hợp cho các bước tiếp theo. 

### Tiến hành xử lí cột `Cause of death`. 
Các bước xử lí:
- Lấy ra dữ liệu của cột `Cause of death`. 
- Sau đó trích xuất ra các giá trị xuất hiện trong cột. 

In [ ]:
cause_of_death_col = raw_df['Cause of death']

cause_of_death_col_unique = cause_of_death_col.unique()

In [ ]:
print(f"số lượng khả năng gây tử vong {len(cause_of_death_col_unique)}")
print(cause_of_death_col_unique)

In [ ]:
cause_count = cause_of_death_col.value_counts()

cause_percentage = ((cause_count / cause_count.sum()) * 100).round(1)

# chuyển thành DataFrame để dễ dàng quan sát. 
cause_df = cause_percentage.reset_index()
cause_df.columns = ["Cause of death", "Percentage"]

# Sắp xếp theo tỷ lệ giảm dần (nếu cần)
cause_df = cause_df.sort_values(by="Percentage", ascending=False)

In [ ]:

sum_of_percentage_greater_than_1 = 0
for i in range(cause_df.shape[0]):
    if cause_df['Percentage'][i] >= 1:
        sum_of_percentage_greater_than_1 += cause_df['Percentage'][i]
        print(f"{cause_df['Cause of death'][i]}: {cause_df['Percentage'][i]}%")


print(f"\ntổng tỷ lệ lớn hơn hoặc bằng 1%: {sum_of_percentage_greater_than_1}%")
print(f"tổng tỷ lệ nhỏ hơn 1%: {100 - sum_of_percentage_greater_than_1}%")

Sau khi quan sát kết quả trên thì ta thấy rằng có thể rút rọn các thông tin lại ít hơn và các tỉ lệ nhỏ hơn 1% ta có thể chuẩn hóa nó thành 1 giá trị riêng là `other cause`. 

In [ ]:
# Bảng ánh xạ tên đầy đủ sang tên rút gọn
cause_mapping = {
    "Extreme immaturity": "Extreme immaturity",
    "Sudden infant death syndrome - SIDS": "SIDS",
    "Other ill-defined and unspecified causes of mortality": "Unspecified causes",
    "Accidental suffocation and strangulation in bed": "Accidental suffocation",
    "Other preterm infants": "Other preterm infants",
    "Newborn affected by premature rupture of membranes": "Premature rupture",
    "Edwards syndrome, unspecified": "Edwards syndrome",
    "Respiratory distress syndrome of newborn": "Respiratory distress",
    "Bacterial sepsis of newborn, unspecified": "Bacterial sepsis",
    "Newborn affected by incompetent cervix": "Incompetent cervix",
    "Newborn affected by chorioamnionitis": "Chorioamnionitis",
    "Necrotizing enterocolitis of newborn": "Necrotizing enterocolitis",
    "Congenital malformation of heart, unspecified": "Congenital heart defect",
    "Neonatal cardiac dysrhythmia": "Cardiac dysrhythmia",
    "Unspecified intraventricular (nontraumatic) haemorrhage of newborn": "Intraventricular haemorrhage",
    "Anencephaly": "Anencephaly",
    "Newborn affected by other forms of placental separation and haemorrhage": "Placental separation",
    "Neonatal cardiac failure": "Cardiac failure",
    "Hypoxic ischemic encephalopathy of newborn": "HIE of newborn"  
}

# Hàm thay thế giá trị, nếu không có trong bảng ánh xạ thì thay thành 'Other causes'
def replace_causes(cause_name):
    return cause_mapping.get(cause_name, "Other causes")

# Áp dụng thay thế trên cột "Cause of death"
raw_df["Cause of death"] = raw_df["Cause of death"].apply(replace_causes)

# Kiểm tra kết quả
print(raw_df["Cause of death"].value_counts())


`Chú ý:` Mặc dù chúng ta đã chuyển đổi dữ liệu về các dạng dễ hơn. Tuy nhiên vẫn có một số bất cập với giá trị `Other cause` vì đây là tổng hợp của các giá trị nhỏ và có thể gần như không có ý nghĩa đối với việc phân tích, Nhưng với số lượng chiếm 4% tổng số (>1000 mẫu) ta có thể cân nhắc việc loại bỏ giá trị này.

In [ ]:
raw_df.head(10)

### Tiến hành xử lí cột `NICU Admission` và `Antibiotics for Mother`.  
Có thể thấy hai cột trên mạng dữ liệu là `Yes` và `No`. Do đó ta sẽ chuyển dữ liệu về dạng số `1` và `0` để có thể dễ dàng xử lí.  

In [ ]:
raw_df['NICU Admission'] = raw_df['NICU Admission'].map({'Yes': 1, 'No': 0})
raw_df['Antibiotics for Mother'] = raw_df['Antibiotics for Mother'].map({'Yes': 1, 'No': 0})

In [ ]:
raw_df.head(5)

### Xử lí cột `Mother's Pre-prehnancy BMI`. 

In [ ]:
mother_pre_col = raw_df["Mother's Pre-pregnancy BMI"]

mother_pre_col_unique = mother_pre_col.unique()
mother_pre_col_unique

Thông qua các giá trị xuất hiện trong cột, ta thực hiện mapping dưới dạng số.

In [ ]:
bmi_mapping = {
    'Underweight <18.5': 0,
    'Normal 18.5-24.9': 1,
    'Overweight 25.0-29.9': 2,
    'Obesity I 30.0-34.9': 3,
    'Obesity II 35.0-39.9': 4,
    'Extreme Obesity III > 39.9': 5,
    'Unknown or Not Stated': -1
}

# Thay thế giá trị trong cột
raw_df["Mother's Pre-pregnancy BMI"] = raw_df["Mother's Pre-pregnancy BMI"].map(bmi_mapping)

### Xử lí cột `Infant birth weight`.

In [ ]:
infant_birth_weight_col = raw_df["Infant birth weight"]

infant_birth_weight_col_unique = infant_birth_weight_col.unique()
infant_birth_weight_col_unique

Hiện tại ta thấy cột trên là biểu diễn về trọng lượng của trẻ khi sinh đo đó ta sẽ chuyển đổi giá trị về giá trị trung bình. 

In [ ]:
def convert_to_mean(weight_str):
    # Xử lý khoảng giá trị (e.g., "500 - 749 grams")
    match = re.match(r"(\d+)\s*-\s*(\d+)", weight_str)
    if match:
        low, high = map(int, match.groups())
        return (low + high) / 2  # Trả về trung bình
    
    # Xử lý "less" hoặc "more" (e.g., "499 grams or less")
    match = re.match(r"(\d+)\s*grams or less", weight_str)
    if match:
        return int(match.group(1)) / 2  # Trả về trung bình
    
    # Nếu không khớp định dạng
    return None

# Áp dụng cho cột "Infant birth weight"
raw_df["Infant birth weight"] = raw_df["Infant birth weight"].apply(convert_to_mean)

### Xử lí cột `Death rate per 1000`. 
- Trong phần này ta sẽ cố gắng loại bỏ các phần `text` trong giá trị của cột. 

In [ ]:
raw_df["Death rate per 1000"] = raw_df["Death rate per 1000"].str.replace(r"\s*\(Unreliable\)", "", regex=True).astype(float)


In [ ]:
raw_df.head(10)

### Kiểm tra lại kiểu dữ liệu của từng cột và chuyển đổi lại nếu cần thiết. 

In [ ]:
raw_df.dtypes

chuyển đổi `Deaths` và `Births` về dạng dữ liệu `int64`

In [ ]:
raw_df['Deaths'] = raw_df['Deaths'].astype('int64')
raw_df['Births'] = raw_df['Births'].astype('int64')

In [ ]:
raw_df.dtypes

### Kiểm tra lại có tồn tại giá trị thiếu (missing value) để tiếp tục xử lí.

In [ ]:
missing_per_column = raw_df.isnull().sum()
print("Số giá trị missing theo từng cột:")
print(missing_per_column)

### lưu dữ liệu vào file để hoàn tất quá trình tiền xử lí (pre-processing)  

In [ ]:
raw_df.to_csv('DATA/infant_mortality_data_2017_2022_cleaned.csv', index=False)

## Explore Data

### Khai báo một số thư viện sử dụng để trực quan dữ liệu

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Ta tiến hành đọc dữ liệu đã được tiền xử lí từ 

In [ ]:
cleaned_df = pd.read_csv('DATA/infant_mortality_data_2017_2022_cleaned.csv')

In [ ]:
cleaned_df.head(10)

### Ta sẽ sử dụng biểu đồ bar plot để biểu diễn tần xuất các giá trị trong từng cột.


In [ ]:
# Thiết lập kích thước mặc định cho các biểu đồ
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Phân phối dữ liệu theo từng cột
def plot_distribution(data, column, title, xlabel):
    plt.figure()
    sns.histplot(cleaned_df[column], kde=True, bins=30, color='blue')
    plt.title(title, fontsize=16)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(True)
    plt.show()

# Lần lượt vẽ phân phối cho các cột số
plot_distribution(cleaned_df, 'Infant birth weight', 'Distribution of Infant Birth Weight', 'Birth Weight (grams)')
plot_distribution(cleaned_df, 'Deaths', 'Distribution of Deaths', 'Number of Deaths')
plot_distribution(cleaned_df, 'Births', 'Distribution of Births', 'Number of Births')
plot_distribution(cleaned_df, 'Death rate per 1000', 'Distribution of Death Rate per 1000', 'Death Rate per 1000')

# Phân phối các cột danh mục đã mã hóa
def plot_categorical_distribution(data, column, title, xlabel):
    plt.figure()
    sns.countplot(x=column, data=data, palette='pastel')
    plt.title(title, fontsize=16)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.grid(axis='y')
    plt.show()

plot_categorical_distribution(cleaned_df, 'NICU Admission', 'Distribution of NICU Admission', 'NICU Admission (1: Yes, 0: No)')
plot_categorical_distribution(cleaned_df, 'Antibiotics for Mother', 'Distribution of Antibiotics for Mother', 'Antibiotics (1: Yes, 0: No)')
plot_categorical_distribution(cleaned_df, "Mother's Pre-pregnancy BMI", "Distribution of Mother's Pre-pregnancy BMI", 'BMI Category')


### Sau khi đã thấy được sự phân bổ giá trị ta vẽ biểu đồ tương quan để quan sát rõ hơn mối tương quan. 

In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = cleaned_df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Matrix', fontsize=16)
plt.show()

- Các biến như `Deaths`, `Births` và `Death rate per 1000` có mỗi quan hệ chặt chẽ với nhau.  
- `Infant birth weight` có mõi quan hệ nghịch với tử vong `Death rate per 1000`. Ta có thể hiểu như sau: Trẻ có cân nặng thấp thường có tỷ lệ tử bong cao hơn. 

### Scatter plot cho hai cột Infant Birth weight và Death Rate

In [ ]:
plt.figure()
sns.scatterplot(x='Infant birth weight', y='Death rate per 1000', hue='NICU Admission', data=cleaned_df, palette='coolwarm', alpha=0.7)
plt.title('Infant Birth Weight vs Death Rate (Colored by NICU Admission)', fontsize=16)
plt.xlabel('Infant Birth Weight (grams)', fontsize=12)
plt.ylabel('Death Rate per 1000', fontsize=12)
plt.grid(True)
plt.legend(title='NICU Admission', labels=['No', 'Yes'])
plt.show()

- Trẻ sơ sinh với cân nặng thấp thường có tỷ lệ tử vong cao hơn, đặc biệt với nhóm không nhập viện như NICU (value = 0).  
- Nhóm nhập viện NICU (value = 1) thường có sự phân tán lớn hơn về tỷ lệ tử vong.  